In [2]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

import re

In [3]:
driver = webdriver.Chrome(executable_path = 
                           'C:\Users\Nathan.y.kelmers\Downloads\chromedriver_win32\chromedriver.exe')
driver.implicitly_wait(2)
driver.get(r'http://personalitycafe.com/forum')

In [4]:
# Logs into Personalitycafe.com
input_loginbox = driver.find_element_by_xpath(r'//*[@id="navbar_username"]')
input_loginbox.send_keys('dataguy', Keys.TAB, 'ds-dc-25', Keys.ENTER)

In [9]:
# Pick the URL of the thread to scrape (do this manually)
threadurl = "http://personalitycafe.com/book-music-movie-reviews/208-what-song-you-listening-now.html"

# Enter how many pages the thread is (do this manually... for now)
pagebegin = 3701
pageend = 3701

In [10]:
mbtidata = []
pageno = pagebegin

for n in range(pagebegin, pageend):
    driver.get(threadurl[0: -5] + "-" + str(pageno) + ".html")
    posts_on_page = []
    
    # Create the soup of of the current page...
    html = driver.page_source
    pagesoup = BeautifulSoup(html)

    # Creates a list containing each post on the page in html form
    posts_on_page = pagesoup.find_all("li", class_ = "postbitlegacy postbitim postcontainer")

    for post in posts_on_page:
        
        # Get post number
        postno = post.find("a", class_ = "postcounter").getText().strip("#")
        
        # Get username
        usersoup = post.find("div", class_ = "userinfo")
        usermenu = usersoup.find("a")
        username = re.search("http://personalitycafe.com/members/(.*).html", str(usermenu)).group(1)

        # Get MBTI
        usertab2 = usersoup.find_all("div", class_ = "tabcontent")[1]
        userinfo = usertab2.find_all("div", class_ = "info")

        if len(userinfo) > 0:
            if "MBTI" in str(userinfo[0]):
                usermbti = str(userinfo[0]).split("</div>")[1]
            else:
                usermbti = None
        else:
            usermbti = None

        # Get messagae
        postsoup = post.find("div", class_ = "postbody")
        postbloc = postsoup.find("blockquote", class_ = "postcontent restore ")
        posttext = re.split("<!-- google_ad_section_start -->|<!-- google_ad_section_end -->", str(postbloc))[1]

        # Append to dataset
        mbtidata.append([postno, username, usermbti, posttext])

    pageno += 1

In [165]:
mbti = pd.DataFrame(mbtidata)
mbti

,0,1,2,3
0,32001,lethal-lava-land,ESFJ,"Otis Redding - ""Down in the Valley""<br/> <br/>..."
1,32002,jippa-jonken,Unknown,"<iframe class=""restrain"" frameborder=""0"" heig..."
2,32003,mr-demiurge,INTP - The Thinkers,"<iframe class=""restrain"" frameborder=""0"" heig..."
3,32004,lethal-lava-land,ESFJ,"The Zombies - ""She's Not There""<br/> <br/>\nTh..."
4,32005,jippa-jonken,Unknown,"<div class=""bbcode_container""> <div class=""bb..."
5,32006,popsthebuilder,None,Black<br/> <br/>\nSevendust<br/> <br/> <br/>\n...
6,32007,toru-okada,Unknown,"Do you like Lily Collins?<br/> <iframe class=""..."
7,32008,aya-the-abysswalker,ENFP,"<iframe class=""restrain"" frameborder=""0"" heig..."
8,32009,cuddlyone,ENFJ,"<iframe class=""restrain"" frameborder=""0"" heig..."
9,32010,benevolentbitterbleeding,ISTJ,"<iframe class=""restrain"" frameborder=""0"" heig..."


In [166]:
mbti.to_csv("C:/Users/Nathan.y.kelmers/Desktop/GA/FINAL PROJECT/mbti_3201-4000.csv")

In [168]:
import glob, os
 
os.chdir("C:\Users\Nathan.y.kelmers\Desktop\GA\FINAL PROJECT\data")
results = pd.DataFrame([])
 
for counter, file in enumerate(glob.glob("mbti_*")):
    namedf = pd.read_csv(file, skiprows = 0)
    results = results.append(namedf)
 
results.to_csv('C:\Users\Nathan.y.kelmers\Desktop\GA\FINAL PROJECT\data\mbti_combined.csv')